In [55]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from sklearn.metrics import confusion_matrix
import itertools

In [42]:
data = pd.read_csv('raw.csv', sep=',',engine='python')
df = data
len(df)

5776

In [43]:
data_new = df.dropna(subset=['ASH','INSUL','TAAV','RH','dayav_ta','dayav_rh','MET'])
data_new.to_csv('raw_new.csv')
# get features and thermal sensation
y = data_new['ASH']
x = data_new[['INSUL','TAAV','RH','dayav_ta','dayav_rh','MET']]
#bool = (y==-3 or y==-2 or y==-1 or y==0 or y==1 or y==2 or y==3)
y = np.round(y)
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y)
np.unique(encoded)
x.iloc[0]

INSUL        0.784200
TAAV        25.000000
RH          61.000000
dayav_ta    27.555556
dayav_rh    81.550000
MET          1.000000
Name: 0, dtype: float64

In [44]:
import sklearn.preprocessing, sklearn.decomposition,sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import datasets, svm
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import RandomForestClassifier

In [45]:
mapper = DataFrameMapper([(['INSUL'], None),
                         (['TAAV'], None),
                          (['RH'], None),
                          (['dayav_ta'], None),
                         (['dayav_rh'],None),
                         (['MET'],None)])
mapper.fit_transform(x.copy())
# count the number of thermal sensation
# bool = (y==3)
# len(y[bool])

array([[  0.7842    ,  25.        ,  61.        ,  27.55555556,
         81.55      ,   1.        ],
       [  0.682944  ,  25.        ,  61.        ,  27.55555556,
         81.55      ,   1.        ],
       [  0.682944  ,  25.        ,  61.        ,  27.55555556,
         81.55      ,   1.        ],
       ..., 
       [  0.82828   ,  23.3       ,  56.1       ,  26.94444444,
         49.5       ,   1.        ],
       [  0.79272   ,  23.2       ,  56.1       ,  26.94444444,
         49.5       ,   1.        ],
       [  0.65324   ,  23.4       ,  53.5       ,  26.94444444,
         49.5       ,   1.        ]])

In [46]:
#clf = svm.SVC(kernel='linear')
C=1
clf = svm.SVC(kernel='rbf',gamma=0.1,C=C)
pipe = sklearn.pipeline.Pipeline([('featurize', mapper),('svc', clf)])
# random forest
clf = RandomForestClassifier(n_estimators=50, max_depth = 3, random_state=0)
pipe = sklearn.pipeline.Pipeline([('featurize', mapper),('randomforest', clf)])
#np.round(cross_val_score(pipe, X=data_new.copy(), y=data_new.comfort, scoring='r2'), 2)
cross_val_score(pipe, X=x.copy(), y=encoded, cv=5)

array([ 0.38145695,  0.41644562,  0.4873502 ,  0.488     ,  0.48796791])

In [47]:
# testing
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

In [53]:
# evaluation
predicted = cross_val_predict(clf, x.copy(), encoded, cv=5)
metrics.accuracy_score(encoded, predicted) 
matrix_sensation = confusion_matrix(encoded, predicted)

In [49]:
test = x.iloc[[0]]
clf.fit(x,y)
gt = y[0]
pred = clf.predict(test)
pred
gt

0.0

In [50]:
import pickle
filename = 'rf_model.sav'
# python2
#pickle.dump(clf,open(filename,'wb'),protocol=2)
# python3
pickle.dump(clf,open(filename,'wb'))
load_model = pickle.load(open(filename,'rb'))
pred = load_model.predict(test)
pred

array([ 0.])

In [57]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [58]:
class_names = {'-3','-2','-1','0','1','2','3'}
plot_confusion_matrix(matrix_sensation,classes=class_names,
                      title='confusion matrix of thermal sensation')

Confusion matrix, without normalization
